<a href="https://colab.research.google.com/github/alfiandz/blog/blob/master/p31202102475.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**P31.2021.02475**
**Muhammad Alfian Dzikri**
```
Text Mining menggunakan preprocessing Sastrawi
dan melakukan pembobotan menggunakan TF-IDF
kemudian ujicoba menggunakan metode Boolean
```



In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
# %matplotlib inline
# pd.set_option('display.max_colwidth', 100)

# Load dataset
def load_data():
    data = pd.read_csv('data_tfidf.csv',sep=';')#ubah nama file sesai dengan nama file kalian
    return data

In [ ]:
paper = load_data()
paper.head()

,jurnal,judul_paper
0,Journal of Biomedical Informatics 1,A comparative analysis of system features used...
1,Journal Information Processing & Management,A case study of distributed information retrie...
2,Journal of Biomedical Informatics 2,Can technological advancements help to allevia...
3,Journal of Biomedical Informatics 3,CDISC-compliant clinical trial imaging managem...
4,Journal Knowledge-Based Systems 1,Complex network modeling of EEG band coupling ...


**Cleaning Data with Sastrawi**

In [ ]:
#definisi dataframe
df  = pd.DataFrame(paper[['judul_paper']])#ubah dataframe sesuai column kalian

#=========================================================================#
#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['judul_paper'], "@[\w]*")
#=========================================================================#

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 30.0 MB/s 


In [ ]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('english')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_jp(jp):
    # remove stock market tickers like $GE
    jp = re.sub(r'\$\w*', '', jp)
 
    # remove old style rejp text "RT"
    jp = re.sub(r'^RT[\s]+', '', jp)
 
    # remove hyperlinks
    jp = re.sub(r'https?:\/\/.*[\r\n]*', '', jp)
    
    # remove hashtags
    # only removing the hash # sign from the word
    jp = re.sub(r'#', '', jp)
    
    #remove coma
    jp = re.sub(r',','',jp)
    
    #remove angka
    jp = re.sub('[0-9]+', '', jp)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    jp_tokens = tokenizer.tokenize(jp)
 
    tweets_clean = []    
    for word in jp_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['jp_clean'] = df['remove_user'].apply(lambda x: clean_jp(x))
#=========================================================================#
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['jp_olah'] = df['jp_clean'].apply(lambda x: remove_punct(x))
 #=========================================================================#

In [ ]:
df.head()

,judul_paper,remove_user,jp_clean,jp_olah
0,A comparative analysis of system features used...,A comparative analysis of system features used...,"[comparative, analysis, system, features, used...",comparative analysis system features used trec...
1,A case study of distributed information retrie...,A case study of distributed information retrie...,"[case, study, distributed, information, retrie...",case study distributed information retrieval a...
2,Can technological advancements help to allevia...,Can technological advancements help to allevia...,"[technological, advancements, help, alleviate,...",technological advancements help alleviate covi...
3,CDISC-compliant clinical trial imaging managem...,CDISC-compliant clinical trial imaging managem...,"[cdisc-compliant, clinical, trial, imaging, ma...",cdisc-compliant clinical trial imaging managem...
4,Complex network modeling of EEG band coupling ...,Complex network modeling of EEG band coupling ...,"[complex, network, modeling, eeg, band, coupli...",complex network modeling eeg band coupling dys...


**TF IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('english')

vectorizer = TfidfVectorizer(stop_words=stopwords_indonesia)
response = vectorizer.fit_transform(df['jp_olah'])

print(response)

  (0, 21)	0.34866645750310943
  (0, 124)	0.27655362219541735
  (0, 76)	0.27655362219541735
  (0, 35)	0.15393009285045026
  (0, 147)	0.34866645750310943
  (0, 151)	0.34866645750310943
  (0, 58)	0.34866645750310943
  (0, 137)	0.306483153027429
  (0, 5)	0.34866645750310943
  (0, 29)	0.34866645750310943
  (1, 141)	0.30958452324767494
  (1, 140)	0.3521946898298045
  (1, 105)	0.3521946898298045
  (1, 75)	0.30958452324767494
  (1, 10)	0.3521946898298045
  (1, 46)	0.3521946898298045
  (1, 135)	0.27935212893128764
  (1, 18)	0.27935212893128764
  (1, 124)	0.27935212893128764
  (1, 76)	0.27935212893128764
  (2, 106)	0.39438849670784704
  (2, 3)	0.44867079533991605
  (2, 66)	0.44867079533991605
  (2, 2)	0.44867079533991605
  (2, 138)	0.44867079533991605
  :	:
  (17, 113)	0.3788429298875782
  (17, 69)	0.3788429298875782
  (17, 122)	0.3788429298875782
  (17, 84)	0.3330087341512015
  (17, 35)	0.16725247329766862
  (18, 62)	0.29430437293776235
  (18, 152)	0.29430437293776235
  (18, 93)	0.2943043729377

In [ ]:
vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'active',
 'advancements',
 'alleviate',
 'ambulance',
 'analysis',
 'analytics',
 'annotated',
 'answering',
 'application',
 'architectures',
 'assessment',
 'assistance',
 'attention',
 'automatic',
 'band',
 'based',
 'brazil',
 'case',
 'categorization',
 'cdisc',
 'challenge',
 'characteristics',
 'characterization',
 'city',
 'clinical',
 'cohort',
 'collection',
 'combating',
 'comparative',
 'complex',
 'compliant',
 'coronavirus',
 'corpus',
 'coupling',
 'covid',
 'creating',
 'dashboard',
 'data',
 'dataset',
 'detection',
 'diagnoses',
 'discovery',
 'disease',
 'dispatch',
 'dissemination',
 'distributed',
 'drug',
 'dyslexia',
 'early',
 'eeg',
 'efficient',
 'ehr',
 'emergency',
 'event',
 'extracting',
 'extraction',
 'fast',
 'features',
 'focusing',
 'framework',
 'gene',
 'genedive',
 'global',
 'graph',
 'health',
 'help',
 'heterogeneity',
 'high',
 'hosts',
 'hybrid',
 'icu',
 'identification',
 'imaging',
 'implementing',
 'index',
 'information',
 '

In [ ]:
response.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.4486708 , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.29430437],
        [0.        , 0.        , 0.        , ..., 0.        , 0.37280988,
         0.        ]])

In [ ]:
import pandas as pd

all_words = []
dict_global = {}

df1 = pd.DataFrame(response.todense().T,
                  index=vectorizer.get_feature_names(),
                  columns=[f'D{i+1}' for i in range(len(df['jp_olah']))])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df1.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20
action,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.323671,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
active,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
advancements,0.0,0.0,0.448671,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
alleviate,0.0,0.0,0.448671,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
ambulance,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.325246,0.0,0.0,0.0,0.0,0.0,0.0


**Boolean**

In [ ]:
df.head()

,judul_paper,remove_user,jp_clean,jp_olah
0,A comparative analysis of system features used...,A comparative analysis of system features used...,"[comparative, analysis, system, features, used...",comparative analysis system features used trec...
1,A case study of distributed information retrie...,A case study of distributed information retrie...,"[case, study, distributed, information, retrie...",case study distributed information retrieval a...
2,Can technological advancements help to allevia...,Can technological advancements help to allevia...,"[technological, advancements, help, alleviate,...",technological advancements help alleviate covi...
3,CDISC-compliant clinical trial imaging managem...,CDISC-compliant clinical trial imaging managem...,"[cdisc-compliant, clinical, trial, imaging, ma...",cdisc-compliant clinical trial imaging managem...
4,Complex network modeling of EEG band coupling ...,Complex network modeling of EEG band coupling ...,"[complex, network, modeling, eeg, band, coupli...",complex network modeling eeg band coupling dys...


In [ ]:
n_headline = 10
dic_new_doc = {}
for i in range(n_headline):
  dic_new_doc['isi jurnal-'+ str(i)] = df.iloc[i,3]

print(dic_new_doc)

{'isi jurnal-0': 'comparative analysis system features used trec-covid information retrieval challenge', 'isi jurnal-1': 'case study distributed information retrieval architectures index one terabyte text', 'isi jurnal-2': 'technological advancements help alleviate covid pandemic', 'isi jurnal-3': 'cdisc-compliant clinical trial imaging management system automatic verification data transformation focusing tumor response assessment data clinical trials', 'isi jurnal-4': 'complex network modeling eeg band coupling dyslexia', 'isi jurnal-5': 'high-throughput method early identification covid presenting symptoms characteristics clinical notes', 'isi jurnal-6': 'covid multidisciplinary data collection dashboard', 'isi jurnal-7': 'creating implementing covid recruitment data mart', 'isi jurnal-8': 'efficient semantic place categorization robot active line-of-sight selection', 'isi jurnal-9': 'novel inverted index fast temporal query application large covid ehr dataset'}


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
import nltk
from nltk.stem import PorterStemmer

In [ ]:
from pandas._libs.hashtable import value_count
pola_angka = r'[0-9]'
stemmer = PorterStemmer()

BOP = {
    'not' : 3,
    'and' : 2,
    'or' : 1,
    '(' : 0,
    ')' : 0,
}
BO = list(BOP.keys())

def preprocessing(teks):
  value = teks.casefold()
  value = re.sub(pola_angka,'',value)
  value = nltk.word_tokenize(value)

  value_stem = []
  for kata in value:
    value_stem.append(stemmer.stem(kata))
  value = value_stem

  return value

def infixtopostfix(query):
  infix = query
  postfix = []
  stack = []
  scanned_operator = ""

  if len(infix) > 0:
    II = 0
    while II < len(infix):
      if infix[II] not in BO:
        postfix.append(infix[II])
        II += 1
      else:
        scanned_operator = infix[II]
        if len(stack) == 0:
          stack.append(scanned_operator)
          II += 1
        elif scanned_operator == "(":
          stack.append(scanned_operator)
          II += 1
        elif scanned_operator == ")":
          postfix.append(stack.pop())
          while len(stack) != 0:
            if stack[-1] != "(":
              postfix.append(stack.pop())
            else:
              stack.pop()
              while len(stack) !=0:
                if stack[-1] !="(":
                  postfix.append(stack.pop())
                else:
                  stack.pop()
              II += 1
          II += 1

        elif BOP[stack[-1]] >= BOP[scanned_operator]:
          postfix.append(stack.pop())
          stack.append(scanned_operator)
          II += 1
        else:
          stack.append(scanned_operator)
          II += 1

    while len(stack) > 0:
      postfix.append(stack.pop())

  return postfix

def booleanMapping(query,dokumen_headline):
  mappedQuery = {}
  BO = ['and', 'or', 'not']
  for headline in dokumen_headline:
    tempList = []
    for item in query:
      if item in BO:
        tempList.append(item)
      else:
        if item in dokumen_headline[headline]:
          tempList.append(True)
        else:
          tempList.append(False)
    mappedQuery[headline] = tempList
  return mappedQuery

def evaluasiBoolean(mappedQuery):
  evaluatedQuery = {}
  BO = ["not", "and", "or"]
  for headline in mappedQuery:
    new_query =mappedQuery[headline]
    while len(new_query) > 1:
      temp_query = []
      for index, kata in enumerate(new_query):
        if kata in BO:
          curr_pos = index
          if kata  == "not":
            ml_pos = curr_pos - 1
            temp_query = temp_query + new_query[:ml_pos]
            temp_query.append(not new_query[ml_pos])
            temp_query = temp_query + new_query[curr_pos+1:]
            new_query = temp_query
            break
          elif kata == "and":
            ml_pos = curr_pos = 2
            mr_pos = curr_pos = 1
            temp_query = temp_query + new_query[:ml_pos]
            temp_query.append(new_query[ml_pos] and new_query[mr_pos])
            temp_query = temp_query + new_query[curr_pos+1:]
            new_query = temp_query
            break
          elif kata == "or":
            ml_pos = curr_pos = 2
            mr_pos = curr_pos = 1
            temp_query = temp_query + new_query[:ml_pos]
            temp_query.append(new_query[ml_pos] or new_query[mr_pos])
            temp_query = temp_query + new_query[curr_pos+1:]
            new_query = temp_query
            break

    evaluatedQuery[headline] = new_query
  return evaluatedQuery

def tampilHasil(evaluatedQuery):
  list_index = []
  list_hasil = []

  for headline in evaluatedQuery:
    if evaluatedQuery[headline][0] == True:
      headline_splited = headline.split(sep='-')
      list_index.append( int (headline_splited[1] ) )
  
  for index in list_index:
    list_hasil.append(df.iloc[index,0])

  return list_hasil

**Pencarian**

In [ ]:
query = input("Pencarian ")

query = preprocessing(query)
query = infixtopostfix(query)
query = booleanMapping(query,dic_new_doc)
query = evaluasiBoolean(query)
hasil = tampilHasil(query)

hasil

Pencarian covid


['A comparative analysis of system features used in the TREC-COVID information retrieval challenge',
 'Can technological advancements help to alleviate COVID-19 pandemic',
 'A high-throughput method for early identification of COVID-19 presenting symptoms and characteristics from clinical notes',
 'COVID-19 multidisciplinary data collection and dashboard',
 'Creating and implementing a COVID-19 recruitment Data Mart',
 'A novel inverted index for fast temporal query, with application to a large Covid-19 EHR dataset']